In [11]:
import os
import cv2
import mediapipe as mp
import json
import numpy as np
import joblib

# Paths
image_folder = r'C:\Users\Istiaque Ahmed\Documents\AIDojo\Keypoint-Based\Final Test\InputImage_and_json'
output_json_path = r'C:\Users\Istiaque Ahmed\Documents\AIDojo\Keypoint-Based\Final Test\InputImage_and_json\classified_keypoints.json'

# Model and Label Encoder paths
model_path = r'C:\Users\Istiaque Ahmed\Documents\AIDojo\Keypoint-Based\Final Test\ModelPreparation\svm_action_classifier.pkl'
label_encoder_path = r'C:\Users\Istiaque Ahmed\Documents\AIDojo\Keypoint-Based\Final Test\ModelPreparation\label_encoder.pkl'

# Load the trained model and label encoder
svm_model = joblib.load(model_path)
label_encoder = joblib.load(label_encoder_path)

# Mediapipe Pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Initialize JSON structure
coco_format = {"images": [], "annotations": []}
image_id = 10000

# Number of keypoints expected by the model
expected_features = 99  # Adjust according to the model's training setup (33 landmarks × 3 attributes)

# Process each image in the folder
for image_filename in os.listdir(image_folder):
    if image_filename.lower().endswith(('.jpeg', '.jpg', '.png')):
        image_path = os.path.join(image_folder, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error reading image: {image_path}")
            continue
        height, width, _ = image.shape

        # Add image metadata
        image_id += 1
        coco_format["images"].append({
            "id": image_id,
            "dataset_id": 10,
            "path": os.path.join("..", "images", image_filename),
            "width": width,
            "height": height,
            "file_name": image_filename
        })

        # Convert BGR image to RGB and process
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_image)

        # Check for landmarks
        if results.pose_landmarks:
            keypoints = []
            # Extract all 33 landmarks
            for lm in results.pose_landmarks.landmark:
                x, y, v = lm.x * width, lm.y * height, lm.visibility
                keypoints.extend([x, y, v])

            # Validate the number of features
            if len(keypoints) != expected_features:
                print(f"Skipping {image_filename}: Extracted {len(keypoints)} features, expected {expected_features}")
                continue

            # Classify keypoints using the trained model
            try:
                keypoints_np = np.array(keypoints).reshape(1, -1)
                predicted_label = svm_model.predict(keypoints_np)[0]
                predicted_class = label_encoder.inverse_transform([predicted_label])[0]
            except Exception as e:
                print(f"Error classifying keypoints for {image_filename}: {e}")
                continue

            # Append annotation data with prediction
            coco_format["annotations"].append({
                "image_id": image_id,
                "keypoints": keypoints,
                "predicted_class": predicted_class
            })

            # Print classification result
            print(f"Image ID: {image_id}, Predicted Class: {predicted_class}")

# Save to JSON file
with open(output_json_path, 'w') as f:
    json.dump(coco_format, f, indent=4)

print(f"Classified keypoints saved to {output_json_path}")
pose.close()


Image ID: 10001, Predicted Class: backhand
Classified keypoints saved to C:\Users\Istiaque Ahmed\Documents\AIDojo\Keypoint-Based\Final Test\InputImage_and_json\classified_keypoints.json


C:\Users\Istiaque Ahmed\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
